In [27]:
import os
import pandas as pd
import fitz ##PyMuPDF
import glob
import copy
from pprint import pprint

In [28]:
pdf_dir = '../pdf/*.pdf'

In [35]:
g = glob.glob( pdf_dir )
fn = g[0]
len(g),g[:3],fn

(38,
 ['../pdf\\CWC2018-19_Leg1_ResultsBook.pdf',
  '../pdf\\CWC2018-19_Leg2_ResultsBook.pdf',
  '../pdf\\CWC2018-19_Leg3_ResultsBook.pdf'],
 '../pdf\\CWC2018-19_Leg1_ResultsBook.pdf')

In [375]:
## ラストストーンをどちらが持っているか?
def solve_laststone(df):
    for i , r in df.iterrows():
        if i == 0:
            continue
        target_value = 0.0
        s = df.loc[df.index == i - 1]    
        if ( (s.red.values[0] == 0) & (s.yellow.values[0] == 0) ): ## ZeroZeroならば、動かない
            target_value = s.last_stone_is_red.values[0]
        else: ## 得点が入れば、点を失ったもの後攻＝　赤の点数が黄色より大きい場合　黄色が後攻
            if (s.red.values[0] < s.yellow.values[0] ):
                target_value = True
            else:
                target_value = False
        df.loc[df.index == i,'last_stone_is_red']= target_value
    return df

def pageToDataFrame(page,num):
    tabs = page.find_tables()
    if len(tabs.tables) <= 0: return None
    ## 1つ目のtableを作る
    scoreTable = tabs.tables[0].extract()
    df = pd.DataFrame(scoreTable)
    is_redHammer = (df[0].values == ['*','']).all()
    is_redYellow = (df[0].values == ['','*']).all()
    assert(any([is_redHammer,is_YellowHammer]))
    df = df.drop(0,axis=1)
    df.index = team
    colname = df.columns.to_list()
    colname = list(map( str , colname))
    colname = colname[:-1] + ['Total']
    df.columns = colname
    df = df.T
    df = df.reset_index()
    df.columns = ['ends','red','yellow']

    def to_number(v):
        if v.isdecimal():return int(v)
        else:return None
    df['red'] = df['red'].map( to_number )
    df['yellow'] = df['yellow'].map( to_number )
    #df = df.dropna(subset=['red','yellow'],axis=0)
    df = df.astype( {'red':float,'yellow':float})

    df['red_diff'] = df['red']
    df.loc[df['yellow'] > 0,'red_diff'] = df.loc[df['yellow'] > 0,'yellow'] * -1
    ## 残end数
    index_v = df.index.to_list()
    df['remain_ends'] = [8-int(v) for v in index_v[:-1]] + [0]
    
    df.loc[df['remain_ends'] < 0 , 'remain_ends'] = 0
    assert(any([is_redHammer,is_YellowHammer]))
    df['last_stone_is_red'] = False
    df.loc[ df.index == 0,'last_stone_is_red' ] = is_redHammer
    df = solve_laststone(df)
    df['page'] = num
    return df


In [366]:
def keyPosion( test_array , key ):
    if not key in test_array: return -1
    else: return test_array.index(key)

In [367]:
page_list = []
doc = fitz.open(fn)  
for i ,page in enumerate(doc):   
    if i < 5 : continue
    text = page.get_text()
    q = text.split('\n')
    key = 'Game Results'  
    title_index = keyPosion( q , key )
    if title_index < 0 :pass
    else:
        page_list.append(i)

In [368]:
for i in page_list:
    page = doc[i]
    break
pageToDataFrame(page)

,ends,red,yellow,red_diff,remain_ends,last_stone_is_red
0,1,0.0,0.0,0.0,8,True
1,2,2.0,0.0,2.0,7,True
2,3,1.0,0.0,1.0,6,False
3,4,0.0,1.0,-1.0,5,False
4,5,1.0,0.0,1.0,4,True
5,6,0.0,2.0,-2.0,3,False
6,7,0.0,0.0,0.0,2,True
7,8,3.0,0.0,3.0,1,True
8,9,NaN,NaN,NaN,0,False
9,Total,7.0,3.0,-3.0,0,False


In [489]:
##l = [pageToDataFrame( doc[i] ) for i in page_list]
l = []
for i in page_list:
    l.append( pageToDataFrame( doc[i], i )  )

In [490]:
len(l)

75

In [491]:
num = page_list.index(330)
dx = l[num] 
dx

,ends,red,yellow,red_diff,remain_ends,last_stone_is_red,page
0,1,2.0,0.0,2.0,8,True,330
1,2,0.0,3.0,-3.0,7,False,330
2,3,2.0,0.0,2.0,6,True,330
3,4,0.0,2.0,-2.0,5,False,330
4,5,2.0,0.0,2.0,4,True,330
5,6,0.0,2.0,-2.0,3,False,330
6,7,0.0,1.0,-1.0,2,True,330
7,8,2.0,0.0,2.0,1,True,330
8,9,0.0,1.0,-1.0,0,False,330
9,Total,8.0,9.0,-9.0,0,True,330


## 画像をあつめる

In [492]:
dx

,ends,red,yellow,red_diff,remain_ends,last_stone_is_red,page
0,1,2.0,0.0,2.0,8,True,330
1,2,0.0,3.0,-3.0,7,False,330
2,3,2.0,0.0,2.0,6,True,330
3,4,0.0,2.0,-2.0,5,False,330
4,5,2.0,0.0,2.0,4,True,330
5,6,0.0,2.0,-2.0,3,False,330
6,7,0.0,1.0,-1.0,2,True,330
7,8,2.0,0.0,2.0,1,True,330
8,9,0.0,1.0,-1.0,0,False,330
9,Total,8.0,9.0,-9.0,0,True,330


In [493]:
outpath = './img/'
def SaveImage( doc ,outpath , game , ends ):
    if ends == 'Total' : 
        return ""
    ends = int(ends)
    page = doc[game+ends]
    text = page.get_text()
    text_array = text.split('\n')
    index = keyPosion(text_array,'Game - Shot by Shot')
    ##print(index)
    if index < 0:
        print(game,ends,'game error : no Shot by Shot')
        return ""
    assert( index > 0 )
    for text in text_array:
        if 'End ' in text:
            page_ends = int(text[-1:])
            #print(page_ends,ends)
            if (page_ends == ends):
                pass
            else:
                print(game,ends,'ends error : no Shot by Shot')
                return ""
            break
    
    cimageList = page.get_images()
    imageId = -1
    for xref in cimageList:
        if xref[7] == 'Im17':
            imageId = xref[0]
            break
    if imageId < 0:
        print(f'game {game}-{ends} no image')
        return ""
    img = doc.extract_image(imageId)
    ext = img['ext']
    path = os.path.join(outpath,f'geme{game}end{ends}.{ext}')
    with open(path, "wb") as ofh:
        ofh.write(img['image'])
    return os.path.basename(path)
    
    

In [505]:
dq = pd.concat(l)
dq = dq.loc[dq['page'] < 450]
dq['filepath'] = dq.apply(lambda r: SaveImage( doc, outpath , r.page , r.ends ) , axis=1)

6 9 game error : no Shot by Shot
19 8 game error : no Shot by Shot
19 9 ends error : no Shot by Shot
27 7 game error : no Shot by Shot
27 8 ends error : no Shot by Shot
27 9 ends error : no Shot by Shot
34 9 game error : no Shot by Shot
43 7 game error : no Shot by Shot
43 8 game error : no Shot by Shot
43 9 ends error : no Shot by Shot
51 8 game error : no Shot by Shot
51 9 ends error : no Shot by Shot
59 9 game error : no Shot by Shot
68 7 game error : no Shot by Shot
68 8 game error : no Shot by Shot
68 9 ends error : no Shot by Shot
76 8 game error : no Shot by Shot
76 9 game error : no Shot by Shot
85 9 game error : no Shot by Shot
94 7 game error : no Shot by Shot
94 8 game error : no Shot by Shot
94 9 ends error : no Shot by Shot
102 7 game error : no Shot by Shot
102 8 ends error : no Shot by Shot
102 9 ends error : no Shot by Shot
109 9 game error : no Shot by Shot
119 7 game error : no Shot by Shot
119 8 ends error : no Shot by Shot
119 9 ends error : no Shot by Shot
126 6 ga

In [506]:
dq.tail()

,ends,red,yellow,red_diff,remain_ends,last_stone_is_red,page,filepath
5,6,2.0,0.0,2.0,3,False,440,geme440end6.png
6,7,NaN,NaN,NaN,2,False,440,
7,8,NaN,NaN,NaN,1,False,440,
8,9,NaN,NaN,NaN,0,False,440,
9,Total,9.0,1.0,-1.0,0,False,440,


In [457]:

img = doc.extract_image(imageId)

In [458]:
ext = img['ext']
outpath = './img/'
path = os.path.join(outpath,f'geme{game}end{ends}.{ext}')
with open(path, "wb") as ofh:
    ofh.write(img['image'])


In [87]:
colname = df.columns.to_list()

In [88]:
colname = list(map( str , colname))

In [89]:
colname = colname[:-1] + ['Total']

In [90]:
colname

['1', '2', '3', '4', '5', '6', '7', '8', '9', 'Total']

In [91]:
df.columns = colname
df

,1,2,3,4,5,6,7,8,9,Total
red,0,2,1,0,1,0,0,3,,7
yellow,0,0,0,1,0,2,0,0,,3


In [93]:
df = df.T

In [95]:
df['isRedLastStone'] = False

In [96]:
df

,red,yellow,isRedLastStone
1,0,0,False
2,2,0,False
3,1,0,False
4,0,1,False
5,1,0,False
6,0,2,False
7,0,0,False
8,3,0,False
9,,,False
Total,7,3,False


In [341]:
page_i = 330
page_i = 231

In [364]:
page = doc[page_i]
tabs = page.find_tables()
##if len(tabs.tables) <= 0: return None
print(len(tabs.tables))
## 1つ目のtableを作る
scoreTable = tabs.tables[0].extract()
df = pd.DataFrame(scoreTable)
is_redHammer = (df[0].values == ['*','']).all()
is_YellowHammer = (df[0].values == ['','*']).all()
assert(any([is_redHammer,is_YellowHammer]))
df = df.drop(0,axis=1)

colname = df.columns.to_list()
colname = list(map( str , colname))
colname = colname[:-1] + ['Total']
df.columns = colname

7


In [350]:
df0 = df.copy()
df = df.T
df = df.reset_index()
df.columns = ['ends','red','yellow']
#df

In [351]:
scoreTable

[['', '0', '3', '0', '2', '0', '1', '0', '0', '', '6'],
 ['*', '1', '0', '1', '0', '1', '0', '1', '1', '', '5']]

In [363]:
df = pd.DataFrame(scoreTable)
is_redHammer = (df[0].values == ['*','']).all()
is_YellowHammer = (df[0].values == ['','*']).all()
is_redHammer , is_YellowHammer


In [345]:
df = df.astype( {'red':int,'yellow':int})

df['red_diff'] = df['red']
df.loc[df['yellow'] > 0,'red_diff'] = df.loc[df['yellow'] > 0,'yellow'] * -1
## 残end数
index_v = df.index.to_list()
df['remain_ends'] = [8-int(v) for v in index_v[:-1]] + [0]

df.loc[df['remain_ends'] < 0 , 'remain_ends'] = 0

ValueError: invalid literal for int() with base 10: '': Error while type casting for column 'red'

In [277]:
df

,ends,red,yellow,red_diff,remain_ends
0,1,2,0,2,8
1,2,0,3,-3,7
2,3,2,0,2,6
3,4,0,2,-2,5
4,5,2,0,2,4
5,6,0,2,-2,3
6,7,0,1,-1,2
7,8,2,0,2,1
8,9,0,1,-1,0
9,Total,8,9,-9,0


In [278]:
assert(any([is_redHammer,is_YellowHammer]))
df['last_stone_is_red'] = False
df.loc[ df.index == 0,'last_stone_is_red' ] = is_redHammer
df

,ends,red,yellow,red_diff,remain_ends,last_stone_is_red
0,1,2,0,2,8,True
1,2,0,3,-3,7,False
2,3,2,0,2,6,False
3,4,0,2,-2,5,False
4,5,2,0,2,4,False
5,6,0,2,-2,3,False
6,7,0,1,-1,2,False
7,8,2,0,2,1,False
8,9,0,1,-1,0,False
9,Total,8,9,-9,0,False


In [432]:
pwd

'C:\\Users\\ghost\\kikagaku\\dataset\\gimage'

In [280]:
df

,ends,red,yellow,red_diff,remain_ends,last_stone_is_red
0,1,2,0,2,8,True
1,2,0,3,-3,7,False
2,3,2,0,2,6,True
3,4,0,2,-2,5,False
4,5,2,0,2,4,True
5,6,0,2,-2,3,False
6,7,0,1,-1,2,True
7,8,2,0,2,1,True
8,9,0,1,-1,0,False
9,Total,8,9,-9,0,True


In [258]:
scoreTable

[['*', '2', '0', '2', '0', '2', '0', '0', '2', '0', '8'],
 ['', '0', '3', '0', '2', '0', '2', '1', '0', '1', '9']]

In [187]:
## チームが取得した点数　or 失った点数
dqw['points'] = dqw['Y']
dqw.loc[ dqw['points'] == 0, 'points'] = dqw['R']  * -1
## 残end数
dqw['remain_ends'] = 10-dqw['ends'] 
dqw.loc[ dqw['remain_ends'] < 0 ,'remain_ends'] = 0
## ラストストーンをどちらが持っているか?
for i , r in dqw.iterrows():
    if i == 0:
        continue
    target_value = 0.0
    s = dqw.loc[dqw['ends'] == (r.ends - 1)]    
    if ( (s.Y.values[0] == 0) & (s.R.values[0] == 0) ): ## ZeroZeroならば、動かない
        target_value = s.last_stone_is_yellow.values[0]
    else: ## 得点が入れば、点を失ったもの後攻＝　赤の点数が黄色より大きい場合　黄色が後攻
        if (s.Y.values[0] < s.R.values[0] ):
            target_value = 1.0
        else:
            target_value = 0.0
    dqw.loc[dqw.index == i,'last_stone_is_yellow']= target_value

1        False
2        False
3         True
4        False
5         True
6        False
7         True
8         True
9        False
Total     True
Name: red_diff, dtype: bool